<a href="https://colab.research.google.com/github/garfield-gray/DataMining/blob/main/peekageHiringChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Peekage Hiring Challenge — Chickapea Campaign

## 0. README

## 1. Imports & Config

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
# Address of files
%cd /content/drive/MyDrive/Life/UniFinance/PeekageHiringChallenge
!ls

/content/drive/MyDrive/Life/UniFinance/PeekageHiringChallenge
 CampaignSubjects.xlsx	   peekageHiringChallenge.ipynb
 ChoiceSubjectsData.xlsx  'Peekage Hiring Challenge.md'


## 2. Data Loading

In [46]:
metaData = pd.read_excel('CampaignSubjects.xlsx')
metaData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             29 non-null     object
 1   SubjectType    29 non-null     object
 2   SubjectName    29 non-null     object
 3   SubjectValues  28 non-null     object
dtypes: object(4)
memory usage: 1.0+ KB


In [47]:
metaData.head()

,Id,SubjectType,SubjectName,SubjectValues
0,"""f36cb6ff-8df4-4842-b59f-d05153bbb560""",SingleSelect,Did you buy Chickapea pasta after trying the s...,"[""No"",""Yes""]"
1,"""d7f7cebb-baa8-41ce-95c5-688f86e50a75""",SingleSelect,Have you ever tried legume-based pasta before?...,"[""No"",""Yes""]"
2,"""80011846-c284-4aa7-9742-8c3f16e131b7""",SingleSelect,How many boxes of Chickapea pasta did you buy?,"[""1"",""2-3"",""4-5"",""More than 5""]"
3,"""3c327491-9c15-4ab3-bdb9-8d2aafbbf87e""",SingleSelect,User Distribution,"[""Activation (Dormant)"",""Acquisition (New-to-b..."
4,"""e0b39b39-682d-462d-ab23-8de2577628fe""",LocationData,Province,"[""Alabama"",""US"",""Alaska"",""US"",""Alberta"",""CA"",""..."


In [48]:
rawData = pd.read_excel('ChoiceSubjectsData.xlsx')
rawData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24255 entries, 0 to 24254
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   RecordId           24255 non-null  object
 1   SamplerId          24255 non-null  object
 2   CampaignSubjectId  24255 non-null  object
 3   SubjectValue       24255 non-null  object
dtypes: object(4)
memory usage: 758.1+ KB


In [49]:
rawData.head()

,RecordId,SamplerId,CampaignSubjectId,SubjectValue
0,"""ac53fafb-7db0-4a5b-a8d6-2c18fab493fb""","""0f98a10f-f296-4808-a7ec-2d20c60bb1cf""","""1450e285-a0bf-4306-bcb0-964b38720b9c""",Kroger
1,"""57e7cbef-0cc4-45cf-9cb3-644ec73b59db""","""ae3f98ed-cee7-4645-80f2-bb4c620e9897""","""1450e285-a0bf-4306-bcb0-964b38720b9c""",Other
2,"""24e414a4-9972-4314-b513-26a52df4eb3d""","""92e80710-705b-48b3-ac62-962005cc254b""","""206cb04a-9519-4623-8109-8b6cf43f224e""",4
3,"""fa99ef64-dcb9-4893-9df4-ce1dadb259da""","""d03f6cf3-5a10-4d9f-8922-6033f3a39688""","""7f3e04bb-5e0d-48f6-a9ac-2f210aec9f97""",Ingredients
4,"""a8daeaa2-d746-4cfb-a3df-570451a2a833""","""4d8be7ea-65a4-427f-8e77-368883fe5193""","""7f3e04bb-5e0d-48f6-a9ac-2f210aec9f97""",Texture


## 3. Data Understanding


In [50]:
rawData.describe()

,RecordId,SamplerId,CampaignSubjectId,SubjectValue
count,24255,24255,24255,24255
unique,24255,1752,29,168
top,"""662a0e45-6d00-4f4f-a880-ae459c2c693f""","""5a1a288c-7655-47c6-b2b8-69293b1b9f95""","""e4eeb58a-5c80-428d-bf24-7372867dd28d""",Yes
freq,1,52,2504,2655


In [51]:
metaData.describe()

,Id,SubjectType,SubjectName,SubjectValues
count,29,29,29,28
unique,29,4,29,21
top,"""f36cb6ff-8df4-4842-b59f-d05153bbb560""",SingleSelect,Did you buy Chickapea pasta after trying the s...,"[""😊"",""😍"",""😐"",""😒""]"
freq,1,20,1,5


In [52]:
rawData.isnull().sum()

,0
RecordId,0
SamplerId,0
CampaignSubjectId,0
SubjectValue,0


In [53]:
metaData.isnull().sum()

,0
Id,0
SubjectType,0
SubjectName,0
SubjectValues,1


In [54]:
# I know it's not wise, but since data is small, I'ma inner join it (denormalizing it:)

metaData['Id'] = metaData['Id'].str.replace('"', '', regex=False)
rawData['CampaignSubjectId'] = rawData['CampaignSubjectId'].str.replace('"', '', regex=False)

merged_df = pd.merge(rawData, metaData, left_on='CampaignSubjectId', right_on='Id', how='inner')

print("Shape of the merged dataframe:", merged_df.shape)
display(merged_df.head())

Shape of the merged dataframe: (23234, 8)


,RecordId,SamplerId,CampaignSubjectId,SubjectValue,Id,SubjectType,SubjectName,SubjectValues
0,"""ac53fafb-7db0-4a5b-a8d6-2c18fab493fb""","""0f98a10f-f296-4808-a7ec-2d20c60bb1cf""",1450e285-a0bf-4306-bcb0-964b38720b9c,Kroger,1450e285-a0bf-4306-bcb0-964b38720b9c,MutliSelect,Where did you buy Chickapea pasta?,"[""Amazon"",""Brands's website"",""Fresh Thyme"",""Ha..."
1,"""57e7cbef-0cc4-45cf-9cb3-644ec73b59db""","""ae3f98ed-cee7-4645-80f2-bb4c620e9897""",1450e285-a0bf-4306-bcb0-964b38720b9c,Other,1450e285-a0bf-4306-bcb0-964b38720b9c,MutliSelect,Where did you buy Chickapea pasta?,"[""Amazon"",""Brands's website"",""Fresh Thyme"",""Ha..."
2,"""24e414a4-9972-4314-b513-26a52df4eb3d""","""92e80710-705b-48b3-ac62-962005cc254b""",206cb04a-9519-4623-8109-8b6cf43f224e,4,206cb04a-9519-4623-8109-8b6cf43f224e,SingleSelect,How do you rate the taste of the pasta?,"[""1"",""2"",""3"",""4"",""5""]"
3,"""fa99ef64-dcb9-4893-9df4-ce1dadb259da""","""d03f6cf3-5a10-4d9f-8922-6033f3a39688""",7f3e04bb-5e0d-48f6-a9ac-2f210aec9f97,Ingredients,7f3e04bb-5e0d-48f6-a9ac-2f210aec9f97,MutliSelect,What did you like the most about Chickapea Spi...,"[""Flavor (Taste)"",""Ingredients"",""Nothing"",""Nut..."
4,"""a8daeaa2-d746-4cfb-a3df-570451a2a833""","""4d8be7ea-65a4-427f-8e77-368883fe5193""",7f3e04bb-5e0d-48f6-a9ac-2f210aec9f97,Texture,7f3e04bb-5e0d-48f6-a9ac-2f210aec9f97,MutliSelect,What did you like the most about Chickapea Spi...,"[""Flavor (Taste)"",""Ingredients"",""Nothing"",""Nut..."


In [55]:
dropped_subject_ids = set(rawData['CampaignSubjectId']) - set(metaData['Id'])
dropped_subject_ids

{'347c0747-787f-4d39-8176-e076649d80fa'}

In [56]:
print(*dropped_subject_ids)

347c0747-787f-4d39-8176-e076649d80fa


In [63]:
dropped_records = rawData[rawData['CampaignSubjectId'].isin(list(dropped_subject_ids))]
print(f"Number of records in rawData with CampaignSubjectId in dropped_subject_ids: {len(dropped_records)}")
display(dropped_records.head())

Number of records in rawData with CampaignSubjectId in dropped_subject_ids: 1021


,RecordId,SamplerId,CampaignSubjectId,SubjectValue
23176,"""19bbb578-b63c-4e07-9a26-38a3d2ef2382""","""ec4f270f-159b-41ff-9b3c-656b1405f8ac""",347c0747-787f-4d39-8176-e076649d80fa,Yes
23235,"""6695a228-fe00-4c3e-b382-dc0f09df1c05""","""d2cca7fc-5df5-43bf-bb9b-52a140a0cf06""",347c0747-787f-4d39-8176-e076649d80fa,Yes
23236,"""c92b21cb-4746-4b02-a4f8-4540da02adc0""","""b4e3a255-fe32-4a34-bf8f-19665819e6ea""",347c0747-787f-4d39-8176-e076649d80fa,Yes
23237,"""15e13be7-afde-4728-9b97-ed9eef1fde36""","""1b142487-1fd2-4493-b2fd-b40e0e944122""",347c0747-787f-4d39-8176-e076649d80fa,Yes
23238,"""b72e8cfd-835c-4cf5-99b4-74eb1586f126""","""855b1773-d7d6-4dbf-8717-e0bc8cef139b""",347c0747-787f-4d39-8176-e076649d80fa,Yes


In [58]:
len(dropped_records)+len(merged_df) == len(rawData)

True

In [62]:
dropped_records.describe()

,RecordId,SamplerId,CampaignSubjectId,SubjectValue
count,1021,1021,1021,1021
unique,1021,1021,1,1
top,"""662a0e45-6d00-4f4f-a880-ae459c2c693f""","""ddb13109-9906-411b-b345-2602d4a255fa""",347c0747-787f-4d39-8176-e076649d80fa,Yes
freq,1,1,1021,1021


In [65]:
dropped_records['SubjectValue'].unique()

array(['Yes'], dtype=object)

In [59]:
metaData[['SubjectType', 'SubjectName', 'SubjectValues']]

,SubjectType,SubjectName,SubjectValues
0,SingleSelect,Did you buy Chickapea pasta after trying the s...,"[""No"",""Yes""]"
1,SingleSelect,Have you ever tried legume-based pasta before?...,"[""No"",""Yes""]"
2,SingleSelect,How many boxes of Chickapea pasta did you buy?,"[""1"",""2-3"",""4-5"",""More than 5""]"
3,SingleSelect,User Distribution,"[""Activation (Dormant)"",""Acquisition (New-to-b..."
4,LocationData,Province,"[""Alabama"",""US"",""Alaska"",""US"",""Alberta"",""CA"",""..."
5,SingleSelect,Gender,"[""Female"",""I would rather not say"",""Male"",""Non..."
6,SingleSelect,Do you have any kids?,"[""Expecting Children"",""No"",""Yes""]"
7,LocationData,Country,"[""CA"",""US""]"
8,SingleSelect,How old are your kids?,"[""0 to 1 years old"",""1 to 3 years old"",""3 to 5..."
9,SingleSelect,Age Range,"[""20-30"",""31-40"",""41-50"",""51-60"",""61-70"",""Unde..."


In [60]:
typeOfData = metaData[['SubjectType', 'SubjectName', 'SubjectValues']]

new_column = {'Type': ['binary', 'binary', 'numeric', 'binary', 'nominal', 'nominal', 'nominal', 'nominal', 'numeric', 'numeric',
                       'set', 'ordinal', 'ordinal', 'ordinal', 'ordinal', 'ordinal', 'numeric', 'numeric', 'ordinal', 'ordinal',
                       'ordinal', 'set', 'numeric', 'binary', 'set', 'set', 'set', 'set', 'string'],  }

typeOfData = typeOfData.assign(**new_column)
typeOfData

,SubjectType,SubjectName,SubjectValues,Type
0,SingleSelect,Did you buy Chickapea pasta after trying the s...,"[""No"",""Yes""]",binary
1,SingleSelect,Have you ever tried legume-based pasta before?...,"[""No"",""Yes""]",binary
2,SingleSelect,How many boxes of Chickapea pasta did you buy?,"[""1"",""2-3"",""4-5"",""More than 5""]",numeric
3,SingleSelect,User Distribution,"[""Activation (Dormant)"",""Acquisition (New-to-b...",binary
4,LocationData,Province,"[""Alabama"",""US"",""Alaska"",""US"",""Alberta"",""CA"",""...",nominal
5,SingleSelect,Gender,"[""Female"",""I would rather not say"",""Male"",""Non...",nominal
6,SingleSelect,Do you have any kids?,"[""Expecting Children"",""No"",""Yes""]",nominal
7,LocationData,Country,"[""CA"",""US""]",nominal
8,SingleSelect,How old are your kids?,"[""0 to 1 years old"",""1 to 3 years old"",""3 to 5...",numeric
9,SingleSelect,Age Range,"[""20-30"",""31-40"",""41-50"",""51-60"",""61-70"",""Unde...",numeric


## 4. Cleaning Rules (Draft)


In [61]:
# declare missing values and null ones !!
# focus more on the aggregation thing


## 5. Metrics Definitions (Draft)


## 6. Agent